# Smart-G demo notebook

This is an interactive document allowing to run Smart-G with python and visualize the results. <br>
*Tips*: cells can be executed with shift-enter. Tooltips can be obtained with shift-tab. More information [here](http://ipython.org/notebook.html) or in the help menu. [A table of content can also be added](https://github.com/minrk/ipython_extensions#table-of-contents).

In [ ]:
%pylab inline
# next 2 lines allow to automatically reload modules that have been changed externally
%reload_ext autoreload
%autoreload 2
import sys
sys.path.append('..')
from smartg import Smartg, reptran_merge
from smartg import RoughSurface, LambSurface, FlatSurface, Environment
from smartg import Profile, AeroOPAC, CloudOPAC, IOP_SPM, IOP_MM, REPTRAN, REPTRAN_IBAND, REPTRAN_IBAND_LIST
from tools.tools import SpecInt, SpecInt2, Irr, ReadREPTRAN_bands, reduce_reptran
from tools.luts import LUT, MLUT, Idx, merge, read_lut_hdf, read_mlut
from tools.smartg_view import plot_polar, smartg_view, input_view, mdesc, transect_view
from ipywidgets import interact, interact_manual
from matplotlib import colors
colors_ = list(six.iteritems(colors.cnames))
# Add the single letter colors.
for name, rgb in six.iteritems(colors.ColorConverter.colors):
    hex_ = colors.rgb2hex(rgb)
    colors_.append((name, hex_))

# Transform to hex color values.
hex_ = [color[1] for color in colors_]
# Get the rgb equivalent.
rgb = [colors.hex2color(color) for color in hex_]

# Split the hsv values to sort.
rr = [color[0] for color in rgb]
gg = [color[1] for color in rgb]
bb = [color[2] for color in rgb]

# Sort by hue, saturation and value.
ind = np.lexsort((rr, gg, bb))
sorted_colors = [colors_[i] for i in ind]

# Equivalence Theorem

In [ ]:
# Here we compare the TOA radiance simulated with
# plane parallel and spherical atmospheres, at 443 nm.
atm=Profile('afglt',grid=np.linspace(100.,0.,num=100), O3=400., NO2=False)
results = []
le={}
le.update(th=np.linspace(0.,89.,num=24)*np.pi/180)
le.update(phi=np.array([0.],dtype=np.float32)*np.pi/180)
for pp in [True, False]:
    S=Smartg(pp=pp, alt_move=True)
    for BEER in [0, 1]:
        results.append(S.run(THVDEG=0., wl=550., NBPHOTONS=1e8, le=le, atm=atm, BEER=BEER))
    
SSA_PP = results[0]['I_up (TOA)']
ET_PP  = results[1]['I_up (TOA)']
SSA_SP = results[2]['I_up (TOA)']
ET_SP  = results[3]['I_up (TOA)']

In [ ]:
((ET_SP-ET_PP)/ET_SP*100.).sub()[0,:].plot(fmt='k-o',vmin=-2,vmax=2)
((SSA_SP-SSA_PP)/SSA_SP*100.).sub()[0,:].plot(fmt='r-o',vmin=-2,vmax=2)
figure()
((SSA_SP-ET_SP)/SSA_SP*100.).sub()[0,:].plot(fmt='r-x',vmin=-.5,vmax=.5)
((SSA_PP-ET_PP)/SSA_PP*100.).sub()[0,:].plot(fmt='k-x',vmin=-.5,vmax=.5)

In [ ]:
# Here we compare the TOA radiance simulated with
# plane parallel and spherical atmospheres, in the EPIC 317 channel for slant view along the day
atm=Profile('afglt',grid=np.linspace(100.,0.,num=100),  NO2=False)
results = []
le={}
le.update(th=np.linspace(0.,89.,num=24)*np.pi/180)
le.update(phi=np.array([0.],dtype=np.float32)*np.pi/180)
for pp in [True, False]:
    S=Smartg(pp=pp, alt_move=True)
    for BEER in [0, 1]:
        results.append(S.run(THVDEG=70., wl=317., NBPHOTONS=1e7, le=le, atm=atm, BEER=BEER))
    
SSA_PP = results[0]['I_up (TOA)']
ET_PP  = results[1]['I_up (TOA)']
SSA_SP = results[2]['I_up (TOA)']
ET_SP  = results[3]['I_up (TOA)']

In [ ]:
((ET_SP-ET_PP)/ET_SP*100.).sub()[0,:].plot(fmt='k-o',vmin=-2,vmax=5)
((SSA_SP-SSA_PP)/SSA_SP*100.).sub()[0,:].plot(fmt='r-o',vmin=-2,vmax=5)
figure()
((SSA_SP-ET_SP)/SSA_SP*100.).sub()[0,:].plot(fmt='r-x',vmin=-.5,vmax=.5)
((SSA_PP-ET_PP)/SSA_PP*100.).sub()[0,:].plot(fmt='k-x',vmin=-.5,vmax=.5)

In [ ]:
SSA_PP = results[0]['I_up (TOA)']
ET_PP  = results[1]['I_up (TOA)']
SSA_SP = results[2]['I_up (TOA)']
ET_SP  = results[3]['I_up (TOA)']

In [ ]:
ET_PP.sub()[0,:].plot(fmt='kx')
ET_SP.sub()[0,:].plot(fmt='ro')

## HITRAN

In [ ]:
import hapi_gpu as hapi

In [ ]:
hapi.db_begin('data')

In [ ]:
def create_profile(z, taur=None, taug=None, taua=None, tauc=None, ssaa=None, ssac=None):
    profile = np.zeros(len(z), dtype=[('I', int),
                                 ('ALT', float),
                                 ('hmol', float),
                                 ('haer', float),
                                 ('H', float),
                                 ('HSCA', float),
                                 ('HABS', float),
                                 ('XDEL', float),
                                 ('YDEL', float),
                                 ('XSSA', float),
                                 ('percent_abs', float),
                                 ('IPHA', int),
                                ])
    
    if taur is None : taur = np.zeros(z.shape)
    if taug is None : taug = np.zeros_like(z)
    if taua is None : taua = np.zeros_like(z)
    if tauc is None : tauc = np.zeros_like(z)
    if ssaa is None : ssaa = np.ones_like(z)
    if ssac is None : ssac = np.ones_like(z)
  
    hray = 0. #cumulated rayl. extinction OT
    haer = 0. #cumulated aero. extinction OT
    hclo = 0. #cumulated clou. extinction OT
    hgas = 0. #cumulated gas   absorption OT
    htot = 0. #cumulated total extinction OT
    hsca = 0. #cumulated total scattering OT
    habs = 0. #cumulated total absorption OT
    
    for m in xrange(len(z)):

        if m==0 :
            profile[m] = (m, z[m], 0., 0., 0., 0., 0., 0., 1., 1., 0., 0)
            
        else :
            hray  += taur[m]
            haer  += taua[m]
            hclo  += tauc[m]
            hgas  += taug[m]
            tau  = taur[m]+ taua[m]+ tauc[m]+ taug[m]
            pabs = taug[m]/tau
           
            if ((taua[m]+tauc[m]) <=0):
                xssa=1.
            else:
                xssa=(ssaa[m]*taua[m] + ssac[m]*tauc[m])/(taua[m]+tauc[m])
                
            tausca = taur[m] + (taua[m]+tauc[m]) * xssa
            ydel = taur[m]/ tausca
            xdel = (taua[m]+tauc[m]) * xssa/ tausca
            hsca += tausca
            
            tauabs = taug[m] + (taua[m]+tauc[m]) * (1.-xssa)
            habs += tauabs
            htot += (tauabs + tausca)
            profile[m] = (m, z[m], hray, haer+hclo, htot , hsca, habs, xdel, ydel, xssa, pabs, 0)

    return profile

### O2

In [ ]:
%%time
dl= 0.001
lmin=765.4
lmax=765.5 + dl

NW=int((lmax-lmin)/dl)+1
vmin=1e7/lmax
vmax=1e7/lmin
dv=(vmax-vmin)/NW
#vmin=12980. #cm-1
#vmax=13200.25 #cm-1
#hapi.fetch('O2i1',7,1,vmin-100,vmax+100)
#hapi.fetch('O2i2',7,2,vmin-100,vmax+100)
#hapi.fetch('O2i3',7,3,vmin-100,vmax+100)
#dv=.25 #cm-1
NL=101
#NW= int((vmax-vmin)/dv)
print 'NW: ', NW
pmin=0.001
pmax=1012.
pnew=np.linspace(pmin,pmax, num=NL)
#znew=abs(-8. *np.log(pnew/pmax))
znew =np.linspace(100.,0.,num=NL)
aer=AeroOPAC('maritime_clean', 0.2, 760.)
atm=Profile('afglmw', grid=znew, O3=0., NO2=False, aer=None, pfwav=[760.],force_cache=True)
dz = np.ediff1d(atm.z, to_begin=0)

ao2=np.zeros((NW,NL),dtype=float)
cao2=np.zeros((NW,NL),dtype=float)
i=0
for p,t,o2,z in zip(atm.P,atm.T,atm.o2,atm.z):
    nuo2,coefo2 = hapi.absorptionCoefficient_Voigt_gpu(SourceTables=['O2i1','O2i2','O2i3'],HITRAN_units=False,
            OmegaRange=[vmin,vmax],OmegaStep=dv,GammaL='gamma_self',
            Environment={'p':p/1013.,'T':t})
    ao2[:,i] = coefo2*1e5
    #ao2[:,i] = 0.
    i=i+1
wl=1e7/nuo2


for i,w in enumerate(wl[::1]):
    
    taur= np.ediff1d(atm.calc(w)['hmol'], to_begin=0)
    
    atm.cache_prof_keys.append(w)
    pro=create_profile(znew, taur=taur, taug=ao2[i,:])
    atm.cache_prof_values.append(pro)                                 

In [ ]:
S=Smartg(alis={'nlow':-1}, double=True)

In [ ]:
S.run(wl=wl,atm=atm,flux='planar', NBPHOTONS=1e8, progress=False)['flux_up (TOA)'].plot(fmt='.-')

### O3

In [ ]:
%%time
dl= 0.1
lmin=280.
lmax=310. + dl

NW=int((lmax-lmin)/dl)
vmin=1e7/lmax
vmax=1e7/lmin
dv=(vmax-vmin)/NW

#hapi.fetch('O3i1',3,1,vmin,vmax)

NL=101
print 'NW: ', NW
pmin=0.001
pmax=1012.
#pnew=np.linspace(pmin,pmax, num=NL)
#znew=abs(-8. *np.log(pnew/pmax))
znew =np.linspace(100.,0.,num=NL)
#aer=AeroOPAC('maritime_clean', 0.2, 760.)
atm=Profile('afglmw', grid=znew, NO2=False, aer=None, pfwav=[300.])
dz = np.ediff1d(atm.z, to_begin=0)

ao3=np.zeros((NW,NL),dtype=float)
cao3=np.zeros((NW,NL),dtype=float)
i=0
for p,t,z in zip(atm.P,atm.T,atm.z):
    nuo3,coefo3 = hapi.absorptionCoefficient_Voigt_gpu(SourceTables=['O3'],HITRAN_units=False,
            OmegaRange=[vmin,vmax],OmegaStep=dv,GammaL='gamma_self',
            Environment={'p':p/1013.,'T':t})
    ao3[:,i] = coefo3*1e5
    i=i+1
wl=1e7/nuo3


for i,w in enumerate(wl[::1]):
    
    taur= np.ediff1d(atm.calc(w)['hmol'], to_begin=0)
    atm.cache_prof_keys.append(w)
    pro=create_profile(znew, taur=taur, taug=ao3[i,:])
    atm.cache_prof_values.append(pro)                                 

In [ ]:
S0=Smartg(alis=False, double=True)
S=Smartg(alis=True, double=True, debug_photon=False)

In [ ]:
##%prun  
le={}
le.update(th=np.array([20.])*np.pi/180)
le.update(phi=np.array([180.])*np.pi/180)
#le={}
#le.update(th=np.linspace(0.,89.,num=12)*np.pi/180)
#le.update(phi=np.linspace(0.,360,num=36)*np.pi/180)
surf=RoughSurface(SUR=1)
surf=None
#m=S.run(THVDEG=60., wl=wl, le=None,  surf=surf, OUTPUT_LAYERS=1, 
        #NBPHOTONS=1e8, atm=atm, BEER=1)
mle1=S.run(THVDEG=30., wl=wl, le=le,  surf=surf, OUTPUT_LAYERS=1, 
        NBPHOTONS=1e6, atm=atm)
#m0=S0.run(THVDEG=0., wl=wl, le=None,  surf=surf,OUTPUT_LAYERS=1, 
          #NBPHOTONS=1e8, atm=atm, BEER=1)
m0le=S0.run(THVDEG=30., wl=wl, le=le,  surf=surf, OUTPUT_LAYERS=1, 
          NBPHOTONS=1e6, atm=atm, BEER=1)
#print m.attrs['kernel time (s)']
print mle1.attrs['kernel time (s)']

In [ ]:
input_view(mle1)
f=figure()
#f.set_size_inches(22,8)
mle1['I_up (TOA)'].sub()[:,0,0].plot(fmt='r.-')
m0le['I_up (TOA)'].sub()[:,0,0].plot(fmt='k-')

# ALIS

## Pure scattering

In [ ]:
S0=Smartg(alis=None, double=True)

In [ ]:
S=Smartg(alis={'nlow':3}, double=True)

In [ ]:
%%time
dl= 50.
lmin=800.
lmax=900.
wref=(lmax-lmin)/2 +lmin
NW=int((lmax-lmin)/dl)+1
wl=np.linspace(lmin,lmax,num=NW)
NL=51
print 'NW: ', NW
znew =np.linspace(100.,0.,num=NL)
aer=AeroOPAC('urban',0.1, wref)
aer=None
#atm=Profile('afglmw', grid=znew, aer=aer, O3=0.,NO2=False, force_cache=False, pfwav=[wref])
atm=Profile('afglmw', grid=znew, aer=aer, O3=0.,NO2=False, force_cache=False)
surf=RoughSurface(SUR=1)
surf=None
le={}
le.update(th=np.array([20.])*np.pi/180)
le.update(phi=np.array([0.])*np.pi/180)
NB=5e7
mle=   S.run(THVDEG=30., wl=wl, le=le, surf=surf, NBPHOTONS=NB, atm=atm)
m0le= S0.run(THVDEG=30., wl=wl, le=le, surf=surf, NBPHOTONS=NB*5, atm=atm, BEER=1)
print mle.attrs['kernel time (s)']

In [ ]:
mle['I_up (TOA)'].sub()[:,0,0].plot(fmt='x-b')
m0le['I_up (TOA)'].sub()[:,0,0].plot(fmt='-xr')
figure()
#mle.describe()
((m0le['I_up (TOA)'].sub()[:,0,0]-mle['I_up (TOA)'].sub()[:,0,0])/(mle['I_up (TOA)'].sub()[:,0,0])).plot(fmt='o-r')

In [ ]:
input_view(mle)

In [ ]:
%%time
dl= 5.
lmin=500.
lmax=650.
NW=int((lmax-lmin)/dl)+1
wl=np.linspace(lmin,lmax,num=NW)
NL=51
print 'NW: ', NW
znew =np.linspace(100.,0.,num=NL)
surf=RoughSurface(SUR=1)
le={}
le.update(th=np.array([20.])*np.pi/180)
le.update(phi=np.array([0.])*np.pi/180)
NB=1e7
atms=[]
for O3 in [180.,200.,220.]:
    atms.append(Profile('afglmw', grid=znew, NO2=False, O3=O3))
    S.run(THVDEG=30., wl=wl, le=le,  DEPO=0., surf=surf, NBPHOTONS=1e5, atm=atm)        

In [ ]:
for atm, col in zip(atms, ['r','g','b']):   
    mle=   S.run(THVDEG=30., wl=wl, le=le, OUTPUT_LAYERS=3,surf=None, NBPHOTONS=NB, atm=atm, progress=False)
    mle['I_down (0+)'].sub()[:,0,0].plot(fmt='.-'+col)
    print mle.attrs['kernel time (s)']

## Jacobian

In [ ]:
%%time
dl= 5.
lmin=500.
lmax=650.
NW=int((lmax-lmin)/dl)+1
print 3*NW
wl=np.linspace(lmin,lmax,num=NW)
_O3 = 300.
O3l=[_O3*0.5, _O3, _O3*1.5]
NL=50.
znew =np.linspace(100.,0.,num=NL)
surf=RoughSurface(SUR=1)
le={}
le.update(th=np.array([20.])*np.pi/180)
le.update(phi=np.array([180.])*np.pi/180)
NB=1e7
atms =Profile('afglmw', grid=znew, NO2=False, O3=0., force_cache=True)
for O3 in O3l:
    atm =Profile('afglmw', grid=znew, NO2=False, O3=O3)
    for w in wl:
        tmp=atm.calc(w)
        taur= np.ediff1d(tmp['hmol'], to_begin=0)
        taug= np.ediff1d(tmp['HABS'], to_begin=0)
        atms.cache_prof_keys.append(w+1e-1)
        pro=create_profile(znew, taur=taur, taug=taug)
        atms.cache_prof_values.append(pro)  

In [ ]:
surf=RoughSurface(SUR=1)
mle1=S.run(THVDEG=30., wl=np.concatenate((wl,wl,wl)), le=le, surf=surf,NBPHOTONS=5e7, atm=atms, progress=False)
print mle1.attrs['kernel time (s)']
print mle1['N_up (TOA)'].data[0][0][0]
mle2=S.run(THVDEG=30., wl=np.concatenate((wl,wl,wl)), le=le, surf=surf,NBPHOTONS=5e7, atm=atms, progress=False)

In [ ]:
Ip1=mle1['Q_up (TOA)']*mle1['Q_up (TOA)']+mle1['U_up (TOA)']*mle1['U_up (TOA)']
DoLP1=(Ip1.apply(np.sqrt)/mle1['I_up (TOA)'])*100.
Ip2=mle2['Q_up (TOA)']*mle2['Q_up (TOA)']+mle2['U_up (TOA)']*mle2['U_up (TOA)']
DoLP2=(Ip2.apply(np.sqrt)/mle2['I_up (TOA)'])*100.
DoLP1.sub()[:,0,0].plot(fmt='b.')
figure()
diff1 =DoLP1.sub()[:,0,0]/DoLP1.sub()[0,0,0]
diff2 =DoLP2.sub()[:,0,0]/DoLP2.sub()[0,0,0]
diff1.plot(fmt='b.')
diff2.plot(fmt='rx')
figure()
(diff1/diff2).plot(fmt='kx')

### O2A jacobian

In [ ]:
%%time
dl= 0.100
lmin=759.5
lmax=769.6
NW=int((lmax-lmin)/dl)
vmin=1e7/lmax
vmax=1e7/lmin
dv=(vmax-vmin)/NW
#vmin=12980. #cm-1
#vmax=13200.25 #cm-1
hapi.fetch('O2i1',7,1,vmin-100,vmax+100)
hapi.fetch('O2i2',7,2,vmin-100,vmax+100)
hapi.fetch('O2i3',7,3,vmin-100,vmax+100)
#dv=.25 #cm-1
NL=101
#NW= int((vmax-vmin)/dv)
print 'NW: ', NW
pmin=0.001
pmax=1012.
pnew=np.linspace(pmin,pmax, num=NL)
#znew=abs(-8. *np.log(pnew/pmax))
znew =np.linspace(100.,0.,num=NL)

NB=1e7
atms =Profile('afglmw', grid=znew, NO2=False, O3=0., force_cache=True)

P0l=[950., 1010.]

ao2=np.zeros((NW,NL),dtype=float)
cao2=np.zeros((NW,NL),dtype=float)
for P0 in P0l:
    i=0
    atm =Profile('afglmw', grid=znew, NO2=False, O3=0., P0=P0)
    for p,t,o2,z in zip(atm.P,atm.T,atm.o2,atm.z):
        nuo2,coefo2 = hapi.absorptionCoefficient_Voigt_gpu(SourceTables=['O2i1','O2i2','O2i3'],HITRAN_units=False,
            OmegaRange=[vmin,vmax],OmegaStep=dv,GammaL='gamma_self',
            Environment={'p':p/1013.,'T':t})
        ao2[:,i] = coefo2*1e5
        #ao2[:,i] = 0.
        i=i+1
    wl=(1e7/nuo2)
    wl=wl[::-1]
   
    for i,w in enumerate(wl):
        tmp=atm.calc(w)
        taur= np.ediff1d(tmp['hmol'], to_begin=0)
        atms.cache_prof_keys.append(w)
        pro=create_profile(znew, taur=taur, taug=ao2[-i,:])
        atms.cache_prof_values.append(pro)  

In [ ]:
S0=Smartg(alis={'nlow':3}, double=True)
le={}
le.update(th=np.linspace(0.,89.,num=10)*np.pi/180)
le.update(phi=np.linspace(0.,360.,num=1)*np.pi/180)

In [ ]:
field='I_up (TOA)'
m=S0.run(THVDEG=30., wl=np.concatenate((wl,wl)),atm=atms, surf=RoughSurface(SUR=1),le=le, NBPHOTONS=1e6, progress=False)
I=m[field].sub()[:NW,0,0]
I.names=['Wavelength']
I.axes[0]=wl
I.desc=mdesc(field)
f0I=I.plot(fmt='.-b')

figure()

jac=(m[field].sub()[NW:2*NW,0,0]-m[field].sub()[0:NW,0,0])/(P0l[-1]-P0l[0])
jac.names=['Wavelength']
jac.axes[0]=wl
jac.desc=r'$d$'+mdesc(field)+r'$/dP_0$'
figure()
f0j=jac.plot(fmt='.-b', vmax=0.00003, vmin=-0.00003)
figure()

In [ ]:
leg=[]
cmap=None; azt=0.; wlt=765.6

for i,(w,(name, color)) in enumerate(zip(wl,colors_)):
    leg.append(str(w))
    if i==0: f=transect_view(m.sub(d={'Wavelength':Idx(w)}), logI=False,ind=Idx(azt), vmin=0, vmax=0.2)
    else: _= transect_view(m.sub(d={'Wavelength':Idx(w)}), ind=Idx(azt), fig=f, color=color, logI=False, vmin=0, vmax=0.2)  
#legend(leg, loc='best')

leg=[]
for i,(az,(name, color)) in enumerate(zip(m.axis('Azimuth angles'),colors_)):
    leg.append(str(az))
    if i==0: f=transect_view(m.sub(d={'Wavelength':Idx(w)}), logI=False,ind=i, vmin=0, vmax=0.2)
    else: _= transect_view(m.sub(d={'Wavelength':Idx(w)}), ind=i, fig=f, color=color, logI=False, vmin=0, vmax=0.2)  
#legend(leg, loc='best') 

leg=[]
for i,(az,(name, color)) in enumerate(zip(m.axis('Azimuth angles'),colors_)):
    leg.append(str(az))
    if i==0: f=transect_view(m.sub(d={'Azimuth angles':i}), swap=True,logI=False,ind=0, sym=False, vmax=0.05)
    else: _= transect_view(m.sub(d={'Azimuth angles':i}), swap=True,ind=0, fig=f, color=color, logI=False, sym=False, vmax=0.05)
#legend(leg, loc='best')

### PARASOL O2

In [ ]:
thv = np.linspace(0., 60. , num=2)
phi = np.linspace(0., 180., num=1)
#thv = np.linspace(0., 60. , num=8)
#phi = np.linspace(0., 180., num=2)
PARASOL_SOLAR = REPTRAN('reptran_solar_parasol')
ibands = PARASOL_SOLAR.to_smartg(lmin=630, lmax=830)
NL = 25; wref = 700.
znew =np.linspace(100.,0.,num=NL)
S = Smartg(alis={'nlow':-1}, double=True)
S0 = Smartg(double=True)
le={}
le.update(th=thv*np.pi/180)
le.update(phi=phi*np.pi/180)

In [ ]:
delta = 0.5
zmin  = 1.
alb=0.1
mult  = np.array([1., 1+delta])
th0   = 30.
COT_l   = 10. * mult
#COT_l   = np.array([1.])
CGT_l   = 2.  * mult
#CGT_l   = np.array([2.])
#REFF_l  = 5.  * mult
REFF_l  = np.array([5.])

surf  = LambSurface(ALB=alb)
atms  = Profile('afglmw', grid=znew, pfwav=[wref], force_cache=True)
for COT in COT_l:
    for CGT in CGT_l:
        for REFF in REFF_l:
            ZMAX = zmin + CGT
            cloud = CloudOPAC('CUMA',[('wc.sol.mie', zmin, ZMAX , 1., REFF)], COT, wref)
            aer = AeroOPAC('desert', COT, wref)
            atm = Profile('afglmw', grid=znew, cloud=cloud, pfwav=[wref], force_cache=False)                           
            #atms.cache_prof_keys.append(ibands.l)
            tmp,_ = atm.calc_bands(ibands.l)
            atms.cache_prof_values.append(tmp)
                
NSIM = len(COT_l)*len(CGT_l)*len(REFF_l)
wl = ibands.l * NSIM    

In [ ]:
m= S.run(THVDEG=th0, wl=wl, NBPHOTONS=1e5, le=le, atm=atm, surf=surf)
print 'time :                            %s'%m.attrs['kernel time (s)']

In [ ]:
#m.describe(show_attrs=True)
print m['tauaer'][:,-1]

In [ ]:
NW=len(ibands.l)
for i in range(NSIM):
    s=reduce_reptran(m.sub(d={'Wavelength':np.arange(i*NW, (i+1)*NW)}), ibands)
    if i==0:
        f1=transect_view(s.sub(d={'Wavelength':0}), field='up (TOA)')
        f2=transect_view(s.sub(d={'Wavelength':1}), field='up (TOA)')
        f3=transect_view(s.sub(d={'Wavelength':2}), field='up (TOA)')
    else:
        _=transect_view(s.sub(d={'Wavelength':0}), field='up (TOA)', fig=f1)
        _=transect_view(s.sub(d={'Wavelength':1}), field='up (TOA)', fig=f2)
        _=transect_view(s.sub(d={'Wavelength':2}), field='up (TOA)', fig=f3)

## Emde et al 2010

In [ ]:
fgas = '/home/mathieu/work/MCCuda/artdeco_output/cTauGas_ray_O2.dat'
gas_valid = np.loadtxt(fgas, skiprows=7)[:,1:]
z_valid   = np.loadtxt(fgas, skiprows=7)[:,0]
w_valid   = np.array(open(fgas).readlines()[5].split()).astype(float)
fray = '/home/mathieu/work/MCCuda/artdeco_output/cTauRay_ray_O2.dat'
ray_valid = np.loadtxt(fray, skiprows=7)[:,1:]
data_valid=np.loadtxt('/home/mathieu/work/MCCuda/resultat/artdeco_lbl_nstr_32_ray_O2.dat')

In [ ]:
atm_valid=Profile('afglmw', grid=z_valid, O3=0., NO2=False, pfwav=[760.],force_cache=True)
for i,w in enumerate(w_valid):    
    taur= np.ediff1d(ray_valid[:,i], to_begin=0) 
    taug= np.ediff1d(gas_valid[:,i], to_begin=0)
    atm_valid.cache_prof_keys.append(w)
    pro=create_profile(z_valid, taur=taur, taug=taug)
    atm_valid.cache_prof_values.append(pro) 
print w_valid.shape

In [ ]:
##%prun  
le={}
le.update(th=np.array([20.])*np.pi/180)
le.update(phi=np.array([180.])*np.pi/180)
mle_valid5=  S.run(THVDEG=30., wl=w_valid, le=le,  DEPO=0., NBPHOTONS=1e9, atm=atm_valid, NBLOOP=10000)
#m0le_valid=S0.run(THVDEG=30., wl=w_valid, le=le,  OUTPUT_LAYERS=1, NBPHOTONS=1e9, atm=atm_valid)
print mle_valid5.attrs['kernel time (s)']

In [ ]:
i_valid=data_valid[:,1]
ip_valid=np.sqrt(data_valid[:,2]*data_valid[:,2]+data_valid[:,3]*data_valid[:,3])
ip=mle_valid5['Q_up (TOA)']*mle_valid5['Q_up (TOA)']+mle_valid5['U_up (TOA)']*mle_valid5['U_up (TOA)']
ip=ip.apply(np.sqrt,mdesc('Ip_up (TOA)'))
i=mle_valid5['I_up (TOA)']
##
plot(w_valid,  i_valid, 'r')
f= i.sub()[:,0,0].plot(fmt='g-', vmax=0.01)
figure()
df=i.sub()[:,0,0]- i_valid
dff=df/i_valid*100
dff.plot(fmt='k-', vmax=3, vmin=-3)
##
figure()
plot(w_valid,  ip_valid, 'r')
f= ip.sub()[:,0,0].plot(fmt='g-', vmax=0.005)
figure()
df=ip.sub()[:,0,0]- ip_valid
dff=df/ip_valid*100
dff.plot(fmt='k-', vmax=3, vmin=-3)

## Tanso FTS

In [ ]:
S=Smartg(alis=True, double=True)

In [ ]:
fgas = '/home/mathieu/work/MCCuda/artdeco_output/cTauGas_ray_O2_fts1.dat'
gas_valid = np.loadtxt(fgas, skiprows=7)[:,1:]
z_valid   = np.loadtxt(fgas, skiprows=7)[:,0]
w_valid   = np.array(open(fgas).readlines()[5].split()).astype(float)
fray = '/home/mathieu/work/MCCuda/artdeco_output/cTauRay_ray_O2_fts1.dat'
ray_valid = np.loadtxt(fray, skiprows=7)[:,1:]
data_valid=np.loadtxt('/home/mathieu/work/MCCuda/resultat/artdeco_lbl_nstr_32_ray_glitter_O2_fts1.dat')
#data_valid=np.loadtxt('/home/mathieu/work/MCCuda/resultat/artdeco_lbl_nstr_32_ray_O2_fts1.dat')

In [ ]:
atm_valid=Profile('afglmw', grid=z_valid, O3=0., NO2=False, pfwav=[760.],force_cache=True)
for i,w in enumerate(w_valid):    
    taur= np.ediff1d(ray_valid[:,i], to_begin=0) 
    taug= np.ediff1d(gas_valid[:,i], to_begin=0)
    atm_valid.cache_prof_keys.append(w)
    pro=create_profile(z_valid, taur=taur, taug=taug)
    atm_valid.cache_prof_values.append(pro)    

In [ ]:
##%prun  
le={}
le.update(th=np.array([20.])*np.pi/180)
le.update(phi=np.array([180.])*np.pi/180)
surf=RoughSurface(SUR=1)
#surf=None
mle_valid_fts1=  S.run(THVDEG=30., wl=w_valid, le=le,  DEPO=0., NBPHOTONS=1e8, 
                       surf=surf, atm=atm_valid, NBLOOP=5000)
print mle_valid_fts1.attrs['kernel time (s)']

In [ ]:
f=figure()
f.set_size_inches(22,8)
mle_valid_fts1['Q_up (TOA)'].sub()[:,0,0].plot(fmt='b-')
mle_valid_fts1['I_up (TOA)'].sub()[:,0,0].plot(fmt='r-', vmax=0.02, vmin=-0.01)

In [ ]:
i_valid=data_valid[:,1]
ip_valid=np.sqrt(data_valid[:,2]*data_valid[:,2]+data_valid[:,3]*data_valid[:,3])
ip=mle_valid_fts1['Q_up (TOA)']*mle_valid_fts1['Q_up (TOA)']+mle_valid_fts1['U_up (TOA)']*mle_valid_fts1['U_up (TOA)']
ip=ip.apply(np.sqrt,mdesc('Ip_up (TOA)'))

i=mle_valid_fts1['I_up (TOA)']

##
plot(w_valid,  i_valid, 'r')
f= i.sub()[:,0,0].plot(fmt='g-', vmax=0.20)
figure()
df=i.sub()[:,0,0]- i_valid
dff=df/i_valid*100
dff.plot(fmt='k-', vmax=5, vmin=-5)
##
ff=figure()
ff.set_size_inches(44,6)
plot(w_valid,  ip_valid, 'r')
f= ip.sub()[:,0,0].plot(fmt='.g', vmax=0.06)
ff.savefig('/home/did/RTC/SMART-G/validation/alis_fts.png',dpi=300)
ff=figure()
ff.set_size_inches(44,6)
df=ip.sub()[:,0,0]- ip_valid
dff=df/ip_valid*100
dff.plot(fmt='.k-', vmax=5, vmin=-5)
ff.savefig('/home/did/RTC/SMART-G/validation/alis_fts_dif.png',dpi=300)

## CO2

In [ ]:
fgas = '/home/mathieu/work/MCCuda/artdeco_output/cTauGas_ray_glitter_CO2.dat'
gas_valid = np.loadtxt(fgas, skiprows=7)[:,1:]
z_valid   = np.loadtxt(fgas, skiprows=7)[:,0]
w_valid   = np.array(open(fgas).readlines()[5].split()).astype(float)
fray = '/home/mathieu/work/MCCuda/artdeco_output/old/Emde_like/cTauRay_ray_glitter_CO2.dat'
ray_valid = np.loadtxt(fray, skiprows=7)[:,1:]
data_valid=np.loadtxt('/home/mathieu/work/MCCuda/resultat/artdeco_lbl_nstr_32_ray_glitter_CO2.dat')

In [ ]:
atm_valid=Profile('afglmw', grid=z_valid, O3=0., NO2=False, pfwav=[760.],force_cache=True)
for i,w in enumerate(w_valid):    
    taur= np.ediff1d(ray_valid[:,i], to_begin=0) 
    taug= np.ediff1d(gas_valid[:,i], to_begin=0)
    atm_valid.cache_prof_keys.append(w)
    pro=create_profile(z_valid, taur=taur, taug=taug)
    atm_valid.cache_prof_values.append(pro)    

In [ ]:
##%prun  
le={}
le.update(th=np.array([20.])*np.pi/180)
le.update(phi=np.array([180.])*np.pi/180)
surf=RoughSurface(SUR=1)
mle_valid_co2=  S.run(THVDEG=30., wl=w_valid, le=le,  DEPO=0., NBPHOTONS=1e8, 
                       surf=surf, atm=atm_valid, NBLOOP=10000)
print mle_valid_co2.attrs['kernel time (s)']

In [ ]:
i_valid=data_valid[:,1]
ip_valid=np.sqrt(data_valid[:,2]*data_valid[:,2]+data_valid[:,3]*data_valid[:,3])
ip=mle_valid_co2['Q_up (TOA)']*mle_valid_co2['Q_up (TOA)']+mle_valid_co2['U_up (TOA)']*mle_valid_co2['U_up (TOA)']
ip=ip.apply(np.sqrt,mdesc('Ip_up (TOA)'))

i=mle_valid_co2['I_up (TOA)']

##
plot(w_valid,  i_valid, 'r')
f= i.sub()[:,0,0].plot(fmt='g-', vmax=0.2)
figure()
df=i.sub()[:,0,0]- i_valid
dff=df/i_valid*100
dff.plot(fmt='k-', vmax=.3, vmin=-.3)
##
figure()
plot(w_valid,  ip_valid, 'r')
f= ip.sub()[:,0,0].plot(fmt='g-', vmax=0.1, vmin=0)
figure()
df=ip.sub()[:,0,0]- ip_valid
dff=df/ip_valid*100
dff.plot(fmt='k-', vmax=.3, vmin=-.3)

## REPTRAN

In [ ]:
thv = np.linspace( 60., 60. , num=1)
phi = np.linspace(180., 180., num=1)
#thv = np.linspace(0., 60. , num=8)
#phi = np.linspace(0., 180., num=2)
FINE_SOLAR = REPTRAN('reptran_solar_fine')
ibands = FINE_SOLAR.to_smartg(lmin=758, lmax=772)
NL = 25; wref = 760.
znew =np.linspace(100.,0.,num=NL)
S = Smartg(alis={'nlow':11}, double=True)
S0 = Smartg(double=True)
le={}
le.update(th=thv*np.pi/180)
le.update(phi=phi*np.pi/180)
atm=Profile('afglt', grid=znew, pfwav=[wref])

In [ ]:
rep=S.run(wl=ibands.l, atm=atm, le=le, NBPHOTONS=1e6, progress=False)

In [ ]:
reduce_reptran(rep,ibands)['I_up (TOA)'].sub()[:,0,0].plot()

In [ ]:
%%time
dl= 0.01
lmin=758
lmax=772.01 + dl

NW=int((lmax-lmin)/dl)
vmin=1e7/lmax
vmax=1e7/lmin
dv=(vmax-vmin)/NW
hapi.fetch('O2i1',7,1,vmin-100,vmax+100)
hapi.fetch('O2i2',7,2,vmin-100,vmax+100)
hapi.fetch('O2i3',7,3,vmin-100,vmax+100)

dz = np.ediff1d(atm.z, to_begin=0)
ao2=np.zeros((NW,NL),dtype=float)
cao2=np.zeros((NW,NL),dtype=float)
i=0
for p,t,o2,z in zip(atm.P,atm.T,atm.o2,atm.z):
    nuo2,coefo2 = hapi.absorptionCoefficient_Voigt_gpu(SourceTables=['O2i1','O2i2','O2i3'],HITRAN_units=False,
            OmegaRange=[vmin,vmax],OmegaStep=dv,GammaL='gamma_self',
            Environment={'p':p/1013.,'T':t})
    ao2[:,i] = coefo2*1e5
    #ao2[:,i] = 0.
    i=i+1
wl=1e7/nuo2
print len(wl)

for i,w in enumerate(wl[::1]):
    
    taur= np.ediff1d(atm.calc(w)['hmol'], to_begin=0)
    
    atm.cache_prof_keys.append(w)
    pro=create_profile(znew, taur=taur, taug=ao2[i,:])
    atm.cache_prof_values.append(pro)                                 

In [ ]:
hitran=S.run(wl=wl,THVDEG=30., atm=atm, le=le, NBPHOTONS=1e6, surf=RoughSurface(), progress=False)
#hitran0=S0.run(wl=wl, atm=atm, le=le, NBPHOTONS=1e7, surf=surf, progress=False)

In [ ]:
hitran['I_up (TOA)'].sub()[:,0,0].plot(fmt='g')
reduce_reptran(rep,ibands)['I_up (TOA)'].sub()[:,0,0].plot(fmt='r')

In [ ]:
I=hitran['I_up (TOA)']
Ip=(hitran['Q_up (TOA)']*hitran['Q_up (TOA)'])+(hitran['U_up (TOA)']*hitran['U_up (TOA)'])
hitran['I_up (TOA)'].sub()[:,0,0].plot(fmt='g-')

## Ocean

In [ ]:
wl = np.linspace(450., 550., num=3)
water = IOP_MM(0.001, pfwav=[500.], DEPTH=100.)
atm=Profile('afglms', pfwav=[500.])
surf=RoughSurface(SUR=3)
thv = np.linspace(60., 60. , num=1)
phi = np.linspace(180.,180., num=1)
S = Smartg(alis={'nlow':-1}, double=True, debug_photon=True)
S0 = Smartg(double=True, debug_photon=False)
le={}
le.update(th=thv*np.pi/180)
le.update(phi=phi*np.pi/180)

In [ ]:
m0=S0.run(wl=wl,THVDEG=50., water=water, surf=None, atm=None, OUTPUT_LAYERS=3,le=le, NBPHOTONS=1e7, progress=False, BEER=0)

In [ ]:
m0.describe()

In [ ]:
m0['I_up (0-)'].sub()[:,0,0].plot()
#m0['I_up (0-)'].sub()[0,:,:].plot_polar(vmax=0.5, index=0)
#m['I_up (TOA)'].sub()[Idx(490.),:,:].plot_polar(vmax=0.1)

In [ ]:
m.plot()